# Scraping data from roaddanger

In [1]:
!pip install selenium

!pip install webdriver-manager

!pip install undetected_chromedriver


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import json
import pandas as pd

In [2]:
# Open and read the JSON file
with open('roaddanger_2.json', 'r', encoding='utf-8') as file:
    data = pd.read_json(file)

# Collect URLs in a list
urls = []

# Extract URLs from the articles
for i in data['articles']:
    for j in i:
        for k in j:
            if k == 'url':
                urls.append(j[k])
                break
        else:
            continue
        break
    else:
        urls.append(None)

print(len(urls))
# Assign the URLs to a new column
data['url'] = urls

# Print the data
print(data.head())
print(data.columns)
print(len(data))

16127
      id                                              title text  \
0  17057      Fietser aangereden op rotonde, Klaphekweg Ede        
1  17049  Auto crasht en slaat over de kop, beknelde bes...        
2  17050  Twee gewonden en flinke schade bij kop-staartb...        
3  17052  Fietser aangereden pal voor voetbalclub Apeldo...        
4  17053     Fiets klem onder auto bij ongeluk in Apeldoorn        

                        date   latitude  longitude countryid  unilateral  pet  \
0  2024-11-25T00:00:00+01:00  52.038747   5.649911        NL           0    0   
1  2024-11-24T00:00:00+01:00        NaN        NaN        NL           1    0   
2  2024-11-24T00:00:00+01:00        NaN        NaN        NL           0    0   
3  2024-11-24T00:00:00+01:00  52.234560   5.981125        NL           0    0   
4  2024-11-24T00:00:00+01:00  52.199836   5.974370        NL           0    0   

   trafficjam                                            persons  \
0           0  [{'id': 43090, 

In [10]:
e_bike_articles = []
bike_articles = []

# Iterate through the data and collect the URLs of the articles that contain words related to bike or e-bike
for id, row in data.iterrows():
    if row['url'] is not None:
        if 'nl' in row['url']:     
            if 'e-bike' in row['title'].lower() or 'fatbike' in row['title'].lower() or 'elektrische fiets' in row['title'].lower():
                e_bike_articles.append(row['url'])
                print(row['url'])
            elif 'fiets' in row['title'].lower():
                bike_articles.append(row['url'])
                print(row['url'])
                
print(len(e_bike_articles))
print(len(bike_articles))


https://www.hardnieuws.nl/gelderland/ede/fietser-aangereden-op-rotonde-klaphekweg-ede-58173.html
https://www.gelderlander.nl/apeldoorn/fietser-aangereden-pal-voor-voetbalclub-apeldoorn-automobilist-rijdt-door~a3a674b2/
https://www.gelderlander.nl/apeldoorn/fiets-klem-onder-auto-bij-ongeluk-in-apeldoorn~ad84c8013/
https://www.ad.nl/wageningen/fietser-gewond-bij-ongeval-op-bronland-in-wageningen~a22fefc0/
https://112ede.nl/weer-aanrijding-op-horror-rotonde#respond
https://www.hardnieuws.nl/gelderland/ede/fietser-gewond-bij-aanrijding-met-personenauto-stadspoort-ede-58091.html
https://www.vrt.be/vrtnws/nl/2024/11/20/zwaar-ongeval-in-diepenbeek-politie-vraagt-om-omgeving-te-vermi/
https://1zwolle.nl/nieuws/152584/graafmachine-valt-uit-vrachtwagen-in-holtenbroek.html
https://www.gelderlander.nl/112-nieuws-veenendaal/fietser-ernstig-gewond-na-botsing-met-touringcar-op-rotonde-in-veenendaal~ae04eac7/
https://www.destentor.nl/112-nieuws-zwolle/fietser-37-overlijdt-in-zwolle-na-aanrijding-met-b

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc



In [8]:
options = webdriver.ChromeOptions() 
driver = uc.Chrome(options=options)

# Initialize the cookies on the websites
driver.get('https://www.ad.nl/')
shadow_host = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "pg-shadow-root-host")))
shadow_root = shadow_host.shadow_root
load = shadow_root.find_element(By.ID, "pg-accept-btn")
load.click()
time.sleep(2)

# Go to the login page
login_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "primary-nav__profile-text"))
)
login_button.click()

# Login with roaddanger credentials :)
input = driver.find_element(By.ID, "username")
input.send_keys("info@digitaldutch.com")

input.send_keys(Keys.RETURN)

password = driver.find_element(By.ID, "password")
password.send_keys("TMYx1KLOUmjDrRb5AnGi")

password.send_keys(Keys.RETURN)

In [14]:
# For every article, collect the date and url and put it into a csv file
urls = []
dates = []

# Check if URL is not yet in e-bike articles csv file 
existing_urls = pd.read_csv('e-bike_articles.csv')['url'].tolist()

for url in e_bike_articles:
    if url not in existing_urls:
        websites = ['https://www.ad.nl/', 'https://www.gelderlander.nl/', 'https://www.ed.nl/', 'https://www.volkskrant.nl/', 'https://www.trouw.nl/', 'https://www.parool.nl/', 'https://www.bndestem.nl/', 'https://bd.nl/', 'https://www.tubantia.nl/', 'https://www.pzc.nl/', 'https://www.destentor.nl/']
        if any(website in url for website in websites):
            urls.append(url)

            # Look for the date of the article (it is in the original dataset json file)
            for id, row in data.iterrows():
                if row['url'] == url:
                    date = row['date']
                    # We are only interested in the date, not the time
                    date = date.split('T')[0]
                    dates.append(date)
                    break
print(len(urls))            
df = pd.read_csv('e-bike_articles.csv')
print(len(df))
df = pd.concat([df, pd.DataFrame({'url': urls, 'date': dates})], ignore_index=True)
df.to_csv('e-bike_articles.csv', index=False)


2
53


In [15]:
# For every article, collect the date and url and put it into a csv file
urls = []
dates = []

# Check if URL is not yet in e-bike articles csv file 
existing_urls = pd.read_csv('bike_articles.csv')['url'].tolist()

for url in e_bike_articles:
    if url not in existing_urls:
        websites = ['https://www.ad.nl/', 'https://www.gelderlander.nl/', 'https://www.ed.nl/', 'https://www.volkskrant.nl/', 'https://www.trouw.nl/', 'https://www.parool.nl/', 'https://www.bndestem.nl/', 'https://bd.nl/', 'https://www.tubantia.nl/', 'https://www.pzc.nl/', 'https://www.destentor.nl/']
        if any(website in url for website in websites):
            urls.append(url)

            # Look for the date of the article (it is in the original dataset json file)
            for id, row in data.iterrows():
                if row['url'] == url:
                    date = row['date']
                    # We are only interested in the date, not the time
                    date = date.split('T')[0]
                    dates.append(date)
                    break
print(len(urls))            
df = pd.read_csv('bike_articles.csv')
print(len(df))
df = pd.concat([df, pd.DataFrame({'url': urls, 'date': dates})], ignore_index=True)
df.to_csv('bike_articles.csv', index=False)


55
747


In [ ]:
articles = []
existing_articles = pd.read_csv('e-bike_articles.csv')['url'].tolist()

# Look for empty article rows in the csv file, where url and date are already filled
for id, row in pd.read_csv('e-bike_articles.csv').iterrows():
    if pd.isnull(row['articles']) and row['url']:

for link in e_bike_articles:
    if link in existing_articles:
        continue
    else:
        # For now only using articles from DPG Media because they have the same layout
        websites = ['https://www.ad.nl/', 'https://www.gelderlander.nl/', 'https://www.ed.nl/', 'https://www.volkskrant.nl/', 'https://www.trouw.nl/', 'https://www.parool.nl/', 'https://www.bndestem.nl/', 'https://bd.nl/', 'https://www.tubantia.nl/', 'https://www.pzc.nl/', 'https://www.destentor.nl/']
        if any(website in link for website in websites):
            driver.get(link)
            time.sleep(5)

            # The alert messages are in a shadow root and need to be closed first
            shadow_host = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "pg-shadow-root-host")))
            shadow_root = shadow_host.shadow_root
            buttons = shadow_root.find_elements(By.CSS_SELECTOR, '.button')
            for button in buttons:
                if button.text == "Melding sluiten":
                    button.click()
                    break

            # Volkskrant, Trouw and Parool have a slightly different layout
            if 'ad.nl' in link or 'gelderlander.nl' in link or 'ed.nl' in link or 'bndestem.nl' in link or 'bd.nl' in link or 'tubantia.nl' in link or 'pzc.nl' in link or 'destentor.nl' in link:
                article_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.CLASS_NAME, 'article'))
                )
            else:
                article_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, 'article-content'))
                )

            # Get the data from the article
            article_element = driver.find_element(By.CLASS_NAME, 'article')
            article_text = article_element.text
            
            if article_text not in existing_articles:
                articles.append(article_text)
                print(article_text)
                time.sleep(2)

# Save the articles to the 'e-bike_articles.csv' file
df = pd.read_csv('e-bike_articles.csv')
df = pd.concat([df, pd.DataFrame({'articles': articles})], ignore_index=True)
df.to_csv('e-bike_articles.csv', index=False)

# Actually 53 articles


In [ ]:
articles = []
for link in bike_articles:
    # For now only using articles from DPG Media because they have the same layout
    websites = ['https://www.ad.nl/', 'https://www.gelderlander.nl/', 'https://www.ed.nl/', 'https://www.volkskrant.nl/', 'https://www.trouw.nl/', 'https://www.parool.nl/', 'https://www.bndestem.nl/', 'https://bd.nl/', 'https://www.tubantia.nl/', 'https://www.pzc.nl/', 'https://www.destentor.nl/']
    if any(website in link for website in websites):
        driver.get(link)
        time.sleep(5)

        # The alert messages are in a shadow root and need to be closed first
        shadow_host = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "pg-shadow-root-host")))
        shadow_root = shadow_host.shadow_root
        buttons = shadow_root.find_elements(By.CSS_SELECTOR, '.button')
        for button in buttons:
            if button.text == "Melding sluiten":
                button.click()
                break


        # Volkskrant, Trouw and Parool have a slightly different layout
        if 'ad.nl' in link or 'gelderlander.nl' in link or 'ed.nl' in link or 'bndestem.nl' in link or 'bd.nl' in link or 'tubantia.nl' in link or 'pzc.nl' in link or 'destentor.nl' in link:
            article_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CLASS_NAME, 'article'))
            )
        else:
            article_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.ID, 'article-content'))
            )

        # Get the data from the article
        article_text = article_element.text
        articles.append(article_text)
        print(article_text)
        time.sleep(2)

        
file_name = "bike_articles.csv"

# Save the articles to the 'bike_articles.csv' file
df = pd.read_csv('bike_articles.csv')
df['articles'] = articles
df.columns = ['url', 'date', 'articles']
df.to_csv('bike_articles.csv', index=False)

# Actually 747 articles

Botsing tussen auto en fietser op Edeseweg in Bennekom. © AS Media
Fietser gewond na aanrijding op gloednieuwe rotonde in Bennekom
Op een nieuwe rotonde op de Edeseweg in Bennekom zijn dinsdagochtend een fietser en een automobilist op elkaar gebotst. De fietser raakte daarbij gewond en werd naar het ziekenhuis gebracht.
112Redactie 05-11-24, 08:28 Laatste update: 05-11-24, 12:45
De botsing gebeurde rond 08.30 uur op dinsdagochtend. Over de oorzaak van het incident is nog niets bekend.
De rotonde ging op 1 november open nadat het kruispunt met stoplichten was omgetoverd tot fietsrotonde in het kader van het Pico Bello pad. Dit pad, dat eind 2025 af moet zijn, vormt straks de snelfietsroute tussen Ede en Wageningen.
Nog even dit…
Wij zijn er voor iedereen die zich betrokken voelt bij onze regio. Wil jij ook graag weten wat er bij ons speelt en wat dat voor jou betekent? Onze verslaggevers zijn de oren en ogen van de regio en staan midden tussen de mensen. Hierdoor zijn wij in staat je he

## Cleaning the data

In [ ]:
# Open the e-bike csv file
df = pd.read_csv('e-bike_articles.csv')

# Open the bike csv file
df2 = pd.read_csv('bike_articles.csv')

# Print the first 5 rows
print(df.head())
print(df2.head())

# Ratio articles about 1:14

                                            articles  \
0  De man raakte op de Dr. Colijnstraat in Veenen...   
1  De persoon op de fatbike werd aan de Rosendaal...   
2  00:51\nTwee meisjes op fatbike hard ten val na...   
3  De fatbiker raakte gewond tijdens het overstek...   
4  De fatbike raakte ligt beschadigd. © Luciano d...   

                                                 url  \
0  https://www.gelderlander.nl/112-nieuws-veenend...   
1  https://www.ad.nl/arnhem/automobilist-botst-op...   
2  https://www.ad.nl/112-nieuws-den-haag/twee-gew...   
3  https://www.ad.nl/ede/fatbiker-gewond-bij-onge...   
4  https://www.ad.nl/112-nieuws-apeldoorn/ongeval...   

                        date  
0  2024-11-04T00:00:00+01:00  
1  2024-11-01T00:00:00+01:00  
2  2024-10-27T00:00:00+02:00  
3  2024-10-27T00:00:00+02:00  
4  2024-10-25T00:00:00+02:00  


In [ ]:
import re

def clean_text(column):
    # Remove the timestamps from the articles
    column = column.str.replace(r'\b\d{2}:\d{2}\b', '', regex=True)
    
    # Remove all the non-alphanumeric characters
    column = column.str.replace(r'[^A-Za-z0-9\s]', ' ', regex=True)   

    # Remove all newline characters
    column = column.str.replace(r'\n', ' ', regex=True)

    # Remove all information from the newspaper itself at the bottom that is not relevant
    column = column.str.replace(r'Nog even dit.*', '', regex=True, flags=re.DOTALL)
    column = column.str.replace(r'Wil je elke dag de Dagelijkse nieuwsupdate nieuwsbrief van AD ontvangen via e mail?.*', '', regex=True, flags=re.DOTALL)
    column = column.str.replace(r'Tip het Parool.*', '', regex=True, flags=re.DOTALL)
    column = column.str.replace(r'Lees ook.*', '', regex=True, flags=re.DOTALL)
    column = column.str.replace(r'Ook interessant voor u.*', '', regex=True, flags=re.DOTALL)

    return column

cleaned_e_bike_articles = clean_text(df['articles'])
cleaned_bike_articles = clean_text(df2['articles'])

print(cleaned_e_bike_articles.head())
print(cleaned_bike_articles.head())

# Save the cleaned articles to the already existing csv file, but in a new column
df['cleaned_articles'] = cleaned_e_bike_articles
df2['cleaned_articles'] = cleaned_bike_articles

df.to_csv('e-bike_articles.csv', index=False)
df2.to_csv('bike_articles.csv', index=False)